In [71]:
from numpy.random import seed
seed(1)
import sklearn
sklearn.__version__
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

import re
from sklearn.metrics import classification_report

In [72]:
data = pd.read_csv('./data/labeled_weather_data.csv', sep="\t")
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23239 entries, 0 to 23238
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   time         23239 non-null  object 
 1   month        23239 non-null  int64  
 2   temperature  23239 non-null  int64  
 3   feelslike    23239 non-null  int64  
 4   wind         23239 non-null  int64  
 5   direction    23239 non-null  object 
 6   gust         23239 non-null  int64  
 7   cloud        23239 non-null  int64  
 8   rain         23239 non-null  float64
 9   rain_ratio   23239 non-null  int64  
 10  pressure     23239 non-null  int64  
 11  weather      23239 non-null  object 
 12  label        23239 non-null  object 
dtypes: float64(1), int64(8), object(4)
memory usage: 2.3+ MB


In [73]:
# Kiểm tra xem đối với trường dữ liệu rain thì có weather như thế nào
condition1 = data['rain'] > 0
condition2 = data['rain'] == 0

# Tính số lượng các giá trị weather đối với rain > 0
result_condition1 = data[condition1]['weather'].value_counts()
# Tính số lượng giá trị weather đối với rain == 0
result_condition2 = data[condition2]['weather'].value_counts()

print("Rain ---------------\n", result_condition1)
print("Norain -------------\n ",result_condition2)


Rain ---------------
 weather
Patchy rain possible              3118
Moderate or heavy rain shower     1034
Moderate rain at times             928
Heavy rain at times                521
Light rain shower                   81
Light drizzle                       38
Light rain                           8
Moderate rain                        4
Patchy light drizzle                 3
Mist                                 3
Patchy light rain with thunder       2
Heavy rain                           2
Patchy light rain                    1
Fog                                  1
Name: count, dtype: int64
Norain -------------
  weather
Clear                          5044
Partly cloudy                  4877
Overcast                       4010
Cloudy                         3438
Patchy rain possible            103
Mist                             19
Thundery outbreaks possible       4
Name: count, dtype: int64


In [74]:
# Xếp loại các loại thời tiết thành 2 dạng là có mưa và không mưa
list_norain = ['Clear', 'Partly cloudy' , 'Overcast','Cloudy', 'Mist',  'Thundery outbreaks possible']
list_rain = ['Patchy rain possible', 'Moderate or heavy rain shower', 'Moderate rain at times', 'Heavy rain at times', 
             'Light rain shower','Light drizzle',  'Light rain', 'Moderate rain', 'Patchy light drizzle',
             'Patchy light rain with thunder', 'Heavy rain', 'Patchy light rain', 'Fog' ]

In [75]:
# Lấy ra cột y để train
# Cột y này là giá trị thời tiết được xếp loại
y = []
numa = 0
numb = 0
for w in data.weather:
    if w in list_norain:
        y+= ['norain']
        numa+=1
    elif w in list_rain:
        y+= ['rain']
        numb+=1
    else:
        y+= ['unknown']
# Do input x có giá trị dự đoán là giá trị thời tiết của 3 giờ tiếp theo
# nên chúng ta xóa dòng đầu tiên
y.pop(0)
print(y)
print(numa)
print(numb)


['norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'rain', 'rain', 'norain', 'rain', 'norain', 'norain', 'norain', 'rain', 'norain', 'norain', 'rain', 'rain', 'rain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'rain', 'norain', 'norain', 'norain', 'norain', 'rain', 'norain', 'norain', 'norain', 'rain', 'rain', 'norain', 'rain', 'rain', 'rain', 'rain', 'norain', 'rain', 'norain', 'norain', 'norain', 'rain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'norain', 'rain', 'rain', 'norain', 'rain', 'norain', 'norain', 'norain', 'rain', 'norain', 'rain', 'rain', 'rain', 'rain', 'rain', 'norain', 'norain', '

In [76]:
# Do input x có giá trị dự đoán là giá trị thời tiết của 3 giờ tiếp theo nhưng không có giá trị đó nên xóa dòng cuối cùng
x = data.drop(23238)


In [77]:
# Tách tập train và tập test với tỉ lệ 9-1
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, stratify = y, random_state=0)

In [78]:
# Tách tập train và tập validation với tỉ lệ 8-2
x_train_v, x_val, y_train_v, y_val = train_test_split(x_train, y_train, test_size = 0.2, stratify = y_train, random_state=0)

In [79]:
# Góm nhóm các giá trị của các cột có kiểu category
# Cột month gom thành 2 giá trị gồm "mua" (5->11) và "kho" (12->4)
# Cột direction thì xóa bỏ kí tự đầu tiên nếu vượt quá 2 ký tự
# Cột weather thì gom nhóm giống như khi gom nhóm y
class ColStd(BaseEstimator, TransformerMixin):
    def fit(self, X_df, y=None):
        return self
    def transform(self, X_df,y=None):
        df = X_df[['time',
                   'month',
                   'temperature',
                   'feelslike',
                   'wind',
                   'direction',
                   'gust',
                   'cloud',
                   'pressure',
                   'weather']]#khong lay cot precipitatinon, rain
        temp_time =[]
        for x in df['time']:
            t = int(re.findall(r'(\d{1,2}):\d{2}',x)[0])
            if ((t>=6) & (t<=15)): 
                temp_time+=['sang']
            else:
                temp_time+=['toi']
        df.time=temp_time
        temp_month =[]
        for x in df['month']:
            if x in [5,6,7,8,9,10]:
                temp_month += ['mua'] 
            elif x in [11,12,1,2,3,4]:
                temp_month += ['kho']
            else:
                temp_month += ['kho']
        df.month = temp_month
        temp_dic = []
        for x in df['direction']:
            if len(x) == 3:
                temp_dic += [x[1:]]
            else:
                temp_dic += [x]
        df.direction = temp_dic
        temp_wea = []
        for w in df['weather']:
            if w in list_norain:
                temp_wea+= ['norain']
            elif w in list_rain:
                temp_wea+= ['rain']
            else:
                temp_wea+= ['unknown']
        df.weather = temp_wea
        return df

In [80]:
# Tạo các Pipeline để tiền xử lý các giá trị có trong dữ liệu
nume_cols = ['temperature', 'feelslike', 'wind', 'gust', 'cloud', 'pressure']
cate_cols = ['time', 'month', 'direction', 'weather']

# Điền giá trị thiếu trong dữ liệu với giá trị trung bình của cột đó
nume_trans = Pipeline(steps=[
    ('imputer1', SimpleImputer(missing_values = np.nan, strategy = 'mean')),
    ('imputer2', SimpleImputer(missing_values = 0, strategy='mean'))
])

# Điền giá trị thiếu trong dữ liệu với giá trị xuất hiện nhiều nhất của cột đó
cate_trans = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy= 'most_frequent')),
    ('onehot', OneHotEncoder(dtype=int, handle_unknown= 'ignore')) # chuyển đổi biến phân loại thành biểu diễn mã hóa one-hot
])

cols_trans = ColumnTransformer(
    transformers=[
        ('nume', nume_trans, nume_cols),
        ('cate', cate_trans, cate_cols)
])

preprocess_pipeline = Pipeline(steps=[
    ('colstd', ColStd()),
    ('coltrans', cols_trans),
    ('std', StandardScaler())
])


In [81]:
# Tiền xử lý tập train x
preprocessed_train_X = preprocess_pipeline.fit_transform(x_train_v)

C:\Users\acer\AppData\Local\Temp\ipykernel_7716\1647454629.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time=temp_time
C:\Users\acer\AppData\Local\Temp\ipykernel_7716\1647454629.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.month = temp_month
C:\Users\acer\AppData\Local\Temp\ipykernel_7716\1647454629.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

In [82]:
# Tiền xử lý tập validation x
preprocess_val_X = preprocess_pipeline.transform(x_val)

C:\Users\acer\AppData\Local\Temp\ipykernel_7716\1647454629.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time=temp_time
C:\Users\acer\AppData\Local\Temp\ipykernel_7716\1647454629.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.month = temp_month
C:\Users\acer\AppData\Local\Temp\ipykernel_7716\1647454629.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

In [83]:
#khảo sát số lượng từng nhãn xem có cân bằng hay không
rain = 0
norain =0
for y in y_train:
    if y =='rain':
        rain+=1
    else:
        norain+=1
print(f"rain: {rain} norain: {norain}" )

rain: 5260 norain: 15654


In [84]:
# Tạo ra full_pipeline bao gồm tiền xử lý và model
mlpclf_model = Pipeline(steps=[
    ('pre', preprocess_pipeline),
    ('mlpclf', MLPClassifier(hidden_layer_sizes=(20),activation='tanh', solver='lbfgs', random_state=0, max_iter=500,))
])


In [125]:
# train và tìm ra các tham số tốt nhất
train_errs = []
val_errs = []
iter = []
alphas = [0.001,0.01, 0.1, 1, 10,20, 100]
hidden_layer_sizes = [1,10,20,50,70,100]

best_val_err = float('inf'); best_alpha = None; best_hls = None;
for alpha in alphas:
    for hls in hidden_layer_sizes:
        iter +=[f"alpha = {alpha}, hls = {hls}"]
        print(iter[-1])
        mlpclf_model.set_params(mlpclf__alpha=alpha,mlpclf__hidden_layer_sizes=(hls))
        mlpclf_model.fit(x_train_v,y_train_v)
        train_err =100*(1 - mlpclf_model.score(x_train_v,y_train_v))
        val_err =100*(1 - mlpclf_model.score(x_val,y_val))

        train_errs += [train_err]
        val_errs +=[val_err]
        if ((val_err < best_val_err)):
            best_val_err = val_err
            best_alpha = alpha
            best_hls = hls


alpha = 0.0001, hls = 1


C:\Users\acer\AppData\Local\Temp\ipykernel_7716\1647454629.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time=temp_time
C:\Users\acer\AppData\Local\Temp\ipykernel_7716\1647454629.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.month = temp_month
C:\Users\acer\AppData\Local\Temp\ipykernel_7716\1647454629.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

alpha = 0.0001, hls = 1


C:\Users\acer\AppData\Local\Temp\ipykernel_7716\1647454629.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time=temp_time
C:\Users\acer\AppData\Local\Temp\ipykernel_7716\1647454629.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.month = temp_month
C:\Users\acer\AppData\Local\Temp\ipykernel_7716\1647454629.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

alpha = 0.0001, hls = 1


...

c:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\acer\AppData\Local\Temp\ipykernel_7716\1647454629.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time=temp_time
C:\Users\acer\AppData\Local\Temp\ipykernel_7716\1647454629.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

alpha = 10, hls = 100


c:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\acer\AppData\Local\Temp\ipykernel_7716\1647454629.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time=temp_time
C:\Users\acer\AppData\Local\Temp\ipykernel_7716\1647454629.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

alpha = 10, hls = 100


c:\Users\acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\acer\AppData\Local\Temp\ipykernel_7716\1647454629.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time=temp_time
C:\Users\acer\AppData\Local\Temp\ipykernel_7716\1647454629.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

In [112]:
# Lưu mô hình
import joblib
joblib.dump(mlpclf_model, './models/mlpclf_model.pkl')

['./models/mlpclf_model.pkl']

In [126]:
for i in range(len(train_errs)):
    print(f"{iter[i]} \n train_err :{train_errs[i]}\t val_err:{val_errs[i]}")
    print()
print(f"best alpha: {best_alpha}")
print(f"best hidden layer size: {best_hls}")

alpha = 0.0001, hls = 1 
 train_err :18.06227960074114	 val_err:17.786277791059046

alpha = 0.0001, hls = 1 
 train_err :18.06227960074114	 val_err:17.786277791059046

alpha = 0.0001, hls = 1 
 train_err :18.06227960074114	 val_err:17.786277791059046

alpha = 0.0001, hls = 10 
 train_err :17.33309425617118	 val_err:17.332058331341138

alpha = 0.0001, hls = 10 
 train_err :17.177694100771024	 val_err:17.834090365766198

alpha = 0.0001, hls = 10 
 train_err :17.219532604147993	 val_err:17.88190294047335

alpha = 0.0001, hls = 20 
 train_err :17.076086306855544	 val_err:17.690652641644753

alpha = 0.0001, hls = 20 
 train_err :16.813101428486043	 val_err:17.547214917523313

alpha = 0.0001, hls = 20 
 train_err :16.77723985416293	 val_err:17.642840066937605

alpha = 0.0001, hls = 50 
 train_err :16.23931623931624	 val_err:18.479560124312698

alpha = 0.0001, hls = 50 
 train_err :15.814954276492744	 val_err:18.766435572555583

alpha = 0.0001, hls = 50 
 train_err :15.372661526507681	 val_er

In [127]:
for i in range(len(train_errs)):
    if (((i+1)%6==0) & (i!=0)):
        print(f"{round(train_errs[i],7)}",end ="\n")
    else:
        print(f"{round(train_errs[i],7)}",end ="\t")

print()
for i in range(len(val_errs)):
    if (((i+1)%6==0 & (i!=0))):
        print(f"{round(val_errs[i],7)}",end ="\n")
    else:
        print(f"{round(val_errs[i],7)}",end ="\t")

18.0622796	18.0622796	18.0622796	17.3330943	17.1776941	17.2195326
17.0760863	16.8131014	16.7772399	16.2393162	15.8149543	15.3726615
16.0779392	15.2590999	14.7630148	15.8448389	14.9303688	14.4163529
18.0802104	18.0802104	18.0802104	17.4466559	17.2852788	17.2793019
17.1657402	16.9983862	16.8131014	16.006216	15.8209312	15.5220848
16.0719622	15.1694459	14.8168071	15.838862	14.9243918	14.3147451
18.0742335	18.0742335	18.0742335	17.267348	17.1418325	17.0282709
16.8250553	16.848963	16.8788476	16.0600084	15.7312773	15.3547307
16.1138007	15.4503616	14.9841611	15.7372542	15.0499074	14.2489989
18.0563027	18.0563027	18.0563027	17.2434403	17.2135557	17.2135557
17.0521786	17.1537864	16.9685016	16.0719622	15.7193234	15.4025462
16.0719622	15.3726615	15.1216305	16.1855239	15.2351922	14.8048533
18.0503257	18.0503257	18.0503257	17.1537864	17.1657402	17.1657402
17.183671	17.189648	17.189648	16.9804554	16.9864324	16.8908015
17.0701094	16.9685016	16.8190784	17.016317	16.9147092	16.8131014

17.7862778	17.786

In [115]:
print(iter)

['alpha = 0.0001, hls = 1', 'alpha = 0.0001, hls = 10', 'alpha = 0.0001, hls = 20', 'alpha = 0.0001, hls = 50', 'alpha = 0.0001, hls = 70', 'alpha = 0.0001, hls = 100', 'alpha = 0.001, hls = 1', 'alpha = 0.001, hls = 10', 'alpha = 0.001, hls = 20', 'alpha = 0.001, hls = 50', 'alpha = 0.001, hls = 70', 'alpha = 0.001, hls = 100', 'alpha = 0.01, hls = 1', 'alpha = 0.01, hls = 10', 'alpha = 0.01, hls = 20', 'alpha = 0.01, hls = 50', 'alpha = 0.01, hls = 70', 'alpha = 0.01, hls = 100', 'alpha = 1, hls = 1', 'alpha = 1, hls = 10', 'alpha = 1, hls = 20', 'alpha = 1, hls = 50', 'alpha = 1, hls = 70', 'alpha = 1, hls = 100', 'alpha = 10, hls = 1', 'alpha = 10, hls = 10', 'alpha = 10, hls = 20', 'alpha = 10, hls = 50', 'alpha = 10, hls = 70', 'alpha = 10, hls = 100', 'alpha = 50, hls = 1', 'alpha = 50, hls = 10', 'alpha = 50, hls = 20', 'alpha = 50, hls = 50', 'alpha = 50, hls = 70', 'alpha = 50, hls = 100', 'alpha = 100, hls = 1', 'alpha = 100, hls = 10', 'alpha = 100, hls = 20', 'alpha = 100,

In [90]:
# Tách alpha và hls  từ iter_data
alpha_values = [float(item.split(',')[0].split('=')[1]) for item in iter]
hls_values  = [float(item.split(',')[1].split('=')[1]) for item in iter]

# Tạo DataFrame từ các mảng
df = pd.DataFrame({
    'alpha_values': alpha_values,
    'hls_values': hls_values ,
    'train_errs1': train_errs,
    'val_errs1': val_errs,
})

# Lưu DataFrame vào tệp Excel (CSV)
df.to_csv('./result/mlpclf_result.csv', index=False)


In [91]:
svmclf_model = Pipeline(steps=[
    ('pre', preprocess_pipeline),
    ('svm', SVC(kernel = 'rbf'))
])

In [92]:
c = [0.1,1,10, 20, 50,100]
gamma = [0.0001,0.001,0.01,0.1,1]
train_errs1 = []
val_errs1 = []
iter1 = []
best_val_err = float('inf'); best_C = None; best_gamma=None

for C in c:
    for g in gamma:
        iter1 +=[f"C={C}, g={g}"]
        print(iter1[-1])
        svmclf_model.set_params(svm__C=C,svm__gamma=g)
        svmclf_model.fit(x_train_v,y_train_v)
        train_err =100*(1 - svmclf_model.score(x_train_v,y_train_v))
        val_err =100*(1 - svmclf_model.score(x_val,y_val))
        train_errs1 += [train_err]
        val_errs1 +=[val_err]
        if ((val_err < best_val_err)):
            best_val_err = val_err
            best_C = C
            best_gamma =g
    

C=0.1, g=0.0001


C:\Users\acer\AppData\Local\Temp\ipykernel_7716\1647454629.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time=temp_time
C:\Users\acer\AppData\Local\Temp\ipykernel_7716\1647454629.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.month = temp_month
C:\Users\acer\AppData\Local\Temp\ipykernel_7716\1647454629.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

C=0.1, g=0.001


C:\Users\acer\AppData\Local\Temp\ipykernel_7716\1647454629.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time=temp_time
C:\Users\acer\AppData\Local\Temp\ipykernel_7716\1647454629.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.month = temp_month
C:\Users\acer\AppData\Local\Temp\ipykernel_7716\1647454629.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

C=0.1, g=0.01


...

C:\Users\acer\AppData\Local\Temp\ipykernel_7716\1647454629.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time=temp_time
C:\Users\acer\AppData\Local\Temp\ipykernel_7716\1647454629.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.month = temp_month
C:\Users\acer\AppData\Local\Temp\ipykernel_7716\1647454629.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

C=100, g=0.1


C:\Users\acer\AppData\Local\Temp\ipykernel_7716\1647454629.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time=temp_time
C:\Users\acer\AppData\Local\Temp\ipykernel_7716\1647454629.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.month = temp_month
C:\Users\acer\AppData\Local\Temp\ipykernel_7716\1647454629.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

C=100, g=1


C:\Users\acer\AppData\Local\Temp\ipykernel_7716\1647454629.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time=temp_time
C:\Users\acer\AppData\Local\Temp\ipykernel_7716\1647454629.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.month = temp_month
C:\Users\acer\AppData\Local\Temp\ipykernel_7716\1647454629.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

In [93]:
# Lưu mô hình
import joblib
joblib.dump(svmclf_model, './models/svm_models.pkl')


['./models/svm_models.pkl']

In [94]:
for i in range(len(train_errs1)):
    print(f"{iter1[i]} \n train_err :{train_errs1[i]}\t val_err:{val_errs1[i]}")
    print()
print(f"best C: {best_C}")
print(f"best gamma: {best_gamma}")

C=0.1, g=0.0001 
 train_err :25.15091745860977	 val_err:25.149414295959836

C=0.1, g=0.001 
 train_err :18.128025820333516	 val_err:17.81018407841263

C=0.1, g=0.01 
 train_err :18.128025820333516	 val_err:17.81018407841263

C=0.1, g=0.1 
 train_err :17.888948658179427	 val_err:17.618933779584033

C=0.1, g=1 
 train_err :21.1762596377981	 val_err:21.730815204398755

C=1, g=0.0001 
 train_err :18.128025820333516	 val_err:17.81018407841263

C=1, g=0.001 
 train_err :18.128025820333516	 val_err:17.81018407841263

C=1, g=0.01 
 train_err :18.128025820333516	 val_err:17.81018407841263

C=1, g=0.1 
 train_err :17.524355985894445	 val_err:17.475496055462592

C=1, g=1 
 train_err :13.830613830613835	 val_err:18.07315323930193

C=10, g=0.0001 
 train_err :18.128025820333516	 val_err:17.81018407841263

C=10, g=0.001 
 train_err :18.128025820333516	 val_err:17.81018407841263

C=10, g=0.01 
 train_err :18.128025820333516	 val_err:17.81018407841263

C=10, g=0.1 
 train_err :16.179546948777723	 val_

In [95]:
for i in range(len(train_errs1)):
    if (((i+1)%5==0 & (i!=0))):
        print(f"{round(train_errs1[i],7)}",end ="\n")
    else:
        print(f"{round(train_errs1[i],7)}",end ="\t")
print()
for i in range(len(val_errs1)):
    if (((i+1)%5==0 & (i!=0))):
        print(f"{round(val_errs1[i],7)}",end ="\n")
    else:
        print(f"{round(val_errs1[i],7)}",end ="\t")

25.1509175	18.1280258	18.1280258	17.8889487	21.1762596
18.1280258	18.1280258	18.1280258	17.524356	13.8306138
18.1280258	18.1280258	18.1280258	16.1795469	7.578746
18.1280258	18.1280258	18.0921642	15.581854	5.7617596
18.1280258	18.1280258	17.8351563	14.8168071	3.8909808
18.1280258	18.1280258	17.6678023	14.2908374	2.8330644

25.1494143	17.8101841	17.8101841	17.6189338	21.7308152
17.8101841	17.8101841	17.8101841	17.4754961	18.0731532
17.8101841	17.8101841	17.8101841	17.4276835	20.4876883
17.8101841	17.8101841	17.7862778	17.4276835	22.0176907
17.8101841	17.8101841	17.5233086	17.9536218	23.9780062
17.8101841	17.8101841	17.1169017	18.383935	24.8864451


In [96]:
print(iter1)

['C=0.1, g=0.0001', 'C=0.1, g=0.001', 'C=0.1, g=0.01', 'C=0.1, g=0.1', 'C=0.1, g=1', 'C=1, g=0.0001', 'C=1, g=0.001', 'C=1, g=0.01', 'C=1, g=0.1', 'C=1, g=1', 'C=10, g=0.0001', 'C=10, g=0.001', 'C=10, g=0.01', 'C=10, g=0.1', 'C=10, g=1', 'C=20, g=0.0001', 'C=20, g=0.001', 'C=20, g=0.01', 'C=20, g=0.1', 'C=20, g=1', 'C=50, g=0.0001', 'C=50, g=0.001', 'C=50, g=0.01', 'C=50, g=0.1', 'C=50, g=1', 'C=100, g=0.0001', 'C=100, g=0.001', 'C=100, g=0.01', 'C=100, g=0.1', 'C=100, g=1']


In [97]:
# Tách c và g từ iter_data
C_values = [float(item.split(',')[0].split('=')[1]) for item in iter1]
g_values = [float(item.split(',')[1].split('=')[1]) for item in iter1]

# Tạo DataFrame từ các mảng
df = pd.DataFrame({
    'C_values': C_values,
    'g_values': g_values,
    'train_errs1': train_errs1,
    'val_errs1': val_errs1,
})

# Lưu DataFrame vào tệp Excel (CSV)
df.to_csv('./result/svm_result.csv', index=False)



In [116]:
#điểm của mô hình mlpclf khi áp dụng tham số tốt nhất dự đoán trên tập tập x_val
mlpclf_model.set_params(mlpclf__alpha=best_alpha,mlpclf__hidden_layer_sizes=best_hls)
mlpclf_model.fit(x_train_v, y_train_v)
y_pred = mlpclf_model.predict(x_val)
print(classification_report(y_val, y_pred))

C:\Users\acer\AppData\Local\Temp\ipykernel_7716\1647454629.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time=temp_time
C:\Users\acer\AppData\Local\Temp\ipykernel_7716\1647454629.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.month = temp_month
C:\Users\acer\AppData\Local\Temp\ipykernel_7716\1647454629.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

              precision    recall  f1-score   support

      norain       0.87      0.91      0.89      3131
        rain       0.69      0.59      0.64      1052

    accuracy                           0.83      4183
   macro avg       0.78      0.75      0.76      4183
weighted avg       0.82      0.83      0.83      4183



C:\Users\acer\AppData\Local\Temp\ipykernel_7716\1647454629.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time=temp_time
C:\Users\acer\AppData\Local\Temp\ipykernel_7716\1647454629.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.month = temp_month
C:\Users\acer\AppData\Local\Temp\ipykernel_7716\1647454629.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

In [101]:
#điểm của mô hình svmclf khi áp dụng tham số tốt nhất dự đoán trên tập tập x_train_v
svmclf_model.set_params(svm__C=best_C,svm__gamma=best_gamma)
svmclf_model.fit(x_train_v,y_train_v)
y_pred = svmclf_model.predict(x_val)

print(classification_report(y_val, y_pred))

C:\Users\acer\AppData\Local\Temp\ipykernel_7716\1647454629.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time=temp_time
C:\Users\acer\AppData\Local\Temp\ipykernel_7716\1647454629.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.month = temp_month
C:\Users\acer\AppData\Local\Temp\ipykernel_7716\1647454629.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

              precision    recall  f1-score   support

      norain       0.88      0.89      0.89      3131
        rain       0.67      0.63      0.65      1052

    accuracy                           0.83      4183
   macro avg       0.77      0.76      0.77      4183
weighted avg       0.83      0.83      0.83      4183



# NHẬN XÉT
- 2 mô hình khá cân băng
- precision của MLP của nhãn rain cao hơn của MLP 0.1
- MLP tốt hơn SVM 1 chút trong việc dự đoán ra nhãn "rain" nên chọn MLP

In [117]:
#huấn luyện trên toàn tập train
mlpclf_model.set_params(mlpclf__alpha=best_alpha,mlpclf__hidden_layer_sizes=best_hls)
mlpclf_model.fit(x_train,y_train)
y_pred = mlpclf_model.predict(x_train)

print(classification_report(y_train, y_pred))

C:\Users\acer\AppData\Local\Temp\ipykernel_7716\1647454629.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time=temp_time
C:\Users\acer\AppData\Local\Temp\ipykernel_7716\1647454629.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.month = temp_month
C:\Users\acer\AppData\Local\Temp\ipykernel_7716\1647454629.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

              precision    recall  f1-score   support

      norain       0.87      0.91      0.89     15654
        rain       0.69      0.58      0.63      5260

    accuracy                           0.83     20914
   macro avg       0.78      0.75      0.76     20914
weighted avg       0.82      0.83      0.82     20914



In [118]:
# Điểm mô hình trên test set
y_pred = mlpclf_model.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

      norain       0.87      0.89      0.88      1740
        rain       0.64      0.59      0.62       584

    accuracy                           0.82      2324
   macro avg       0.76      0.74      0.75      2324
weighted avg       0.81      0.82      0.81      2324



C:\Users\acer\AppData\Local\Temp\ipykernel_7716\1647454629.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.time=temp_time
C:\Users\acer\AppData\Local\Temp\ipykernel_7716\1647454629.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.month = temp_month
C:\Users\acer\AppData\Local\Temp\ipykernel_7716\1647454629.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

# Nhận xét
- Độ chính xác của mô mình đạt được mong đợi (>80%)
- Mô hình dự đoán tốt đối với nhãn không mưa (f1-score : 0.88)
- Mô hình dự đoán nhãn mưa không tốt lắm, nhưng cũng không hề tệ tuy nhiên độ hiệu quả cũng tương đối


# Kết luận
- Dự đoán là khi áp dụng vào thực tế, mô hình hoạt động sẽ không tốt, nhưng không quá tệ
- Có thể sử dụng làm nguồn tham khảo nếu không thể xem dự báo thời tiết vì một lý do nào đó 